# Aplicando web scraping a la página UniProt

## Presentado por:
- Campuzano Galarza, Sofia Gabriela
- Cordova Quispe, Brigitte Nayely
- Tejada Flores, Antonella Franchesca

## Objetivo
Extraer información de la entrada (Entry), nombre de entrada (Entry name), proteína (Protein), gen (Gene), organismo (Organism), estado (Status) y GO-Proceso biológico (GO-Biological Process) de los más de seis mil genes abreviados (Gen Abrev) para posteriormente almacenarlos en un archivo excel.


## ¿La página tiene un API?
Tras una intensa busqueda, realizada por el grupo, pudimos validar que no existe API alguno para esta página. Por lo que, obtamos por inspeccionar de manera manual el html de la página y luego con ayuda del paquete BeautifulSoup pudimos extraer la data solicitada en el archivo Excel_UniProt_Vacio.xlsx.

### Nota:
- Se recomienda guardar el archivo ipynb y el excel vacío en la misma carpeta para que no haya problemas al ejecutar el código. 

- Este código esta trabajando con un archivo excel de nombre "02 Ch Genes ordenados con vecinos_PROTOTIPO.xlsx", ya que contiene una muestra de 16 genes. Si se desea correr todos los genes que existen en la página de **UniProt.org**, cambie el nombre del archivo excel por el de "Excel_UniProt_Vacio.xlsx". Debe realizar los cambios en la línea 5, 103 y 114.

### Link de la página:
https://www.uniprot.org


In [ ]:
                    ########################### PAQUETES A IMPORTAR ###########################
import requests
import openpyxl
import pandas as pd
from bs4 import BeautifulSoup

                ################## CÓDIGO INPLEMENTADO PARA EL WEB SCRAPING  ######################
    
archivo_vacio = pd.read_excel("02 Ch Genes ordenados con vecinos_PROTOTIPO.xlsx",header=1) # Lee el archivo excel que se tiene guardada en la misma carpeta del script.
genes=list(archivo_vacio["Gen Abrev"]) # De la variable archivo_vacio que guarda la información del excel,
# se extre la columna "Gen Abrev" para su posterior uso en la iteración a la hora de buscar gen por gen.


n=3 # variable que itera y ubica la fila de genes en el archivo excel.
for i in genes: # Por cada elemento en la lista de genes:
    print(n-2,":",i)
    url=f"https://www.uniprot.org/uniprot/?query={i}+Capra+Hircus&sort=score" # Guarda la url que va iterando cada fila de los elementos de la columna "Gen Abrev". 
    page_response=requests.get(url) # Hace una "solicitud" a la página del tipo "obtener" para extraer la información
    # de la dirección de url.
    page_content=BeautifulSoup(page_response.content,"html.parser") # Analiza la respuesta del "page.response" y lo guarda como
    # un html para que pueda recorrer por el código fuente.
    
    
    # Con el fin de entrar a la página de cada gen para extraer la información solicitado,
    # Se realiza lo siguiente:

    entry=page_content.find_all("td",class_="entryID") # Encuentra todos las etiquetas  "td" con las clase "entryID".
    entry_i=[] # Lista vacía para almacenar la información que esta guardada en la variable entry.
    for e in entry:  # Por cada elemento en entry:
        entry_i.append(e.text) # Añade a la lista vacía solo el texto que encuente "entry".
        
        
    # Utilizamos "try" porque a la hora de iterar buscando página por página y no encuentre nada de información en un gen,
    # se pase a la búsqueda de otro gen. De esta manera evitamos errores en la extracción de información. 
        
    try: # Intenta: 
        url=f"https://www.uniprot.org/uniprot/{entry_i[0]}" # Guarda la url de la página de cada gen, que se itera por el texto extraído de la primera fila de "entry_i"
        page_response=requests.get(url) # Realiza la "solicitud" a la página del tipo "obtener" para extraer la información
        # de la dirección url.
        page_content=BeautifulSoup(page_response.content,"html.parser") # Analiza la respuesta del "page.response" y
        # lo guarda como un html para que pueda recorrer por el código fuente.
            
            
        ####  ENTRY  ####
        entry_name=page_content.find_all("h2",class_="page-title") # Busca todas la etiquetas "h2" acompañada de la clase "page-title"
        for j in entry_name: # Para cada elemento en entry_name:
            entry1=j.text # extrae el texto que se encuentre en los elementos buscados de entry_name.
        entry1=entry1.split() # Se utiliza la función split para separa los elementos de la lista.
        Entry=entry1[2] # Como la lista se cuenta a partir del 0, se selecciona el elemento 2 de la lista entry1.  
        
        
        #### ENTRY NAME ####
        Entry_name="".join(list(entry1[3])[1:len(list(entry1[3]))-1])
        # De la lista entry1 selecciona el elemento 3, dicho elemento lo pasa a una lista, de dicha lista nueva 
        # selecciona a partir de la posición (1) toda la longitud hasta la ubicación (-1). De esta manera se evita las paréntesis.  
        
        
        #### PROTEIN ####
        proteina=page_content.find_all("h1",property="name") 
        for j in proteina:
            Protein=j.text

        #### GEN ####
        gabrev=page_content.find_all("div",class_='entry-overview-content',id="content-gene")
        for j in gabrev: 
            Gen_Abrev=j.text 
            
       #### ORGANISMO ####
        organism_gen=page_content.find_all("div",class_='entry-overview-content',id="content-organism")
        for j in organism_gen:
            ORGANISM_GEN=j.text
        
        #### STATUS ####
        estado=page_content.find_all("a",title="Unreviewed (TrEMBL)", class_="icon-uniprot unreviewed-icon tooltipped")
        for j in estado:
            ESTADO_PREVIO=j.text
        ESTADO=str(ESTADO_PREVIO)

    except:
        Entry =""
        Entry_name =""
        Protein =""
        Gen_Abrev =""
        ORGANISM_GEN =""
        ESTADO =""
    
    try:
        url=f"https://www.uniprot.org/uniprot/{entry_i[0]}"
        page=requests.get(url)
        page_content=BeautifulSoup(page.content,"html.parser")
        
        #### GO BIOLOGICAL PROCESS ####  
        go_secuencia = page_content.find("ul", attrs= {"class":"noNumbering biological_process"})
        for secuencia in go_secuencia:  
            pro_biological_info = secuencia.find_all("a", attrs={"onclick":"window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');"})
            for j in range(len(pro_biological_info)):
                GO_PROCESO.append(pro_biological_info[j].text)
        GO_PROCESO=str(GO_PROCESO)
        
    except: 
        GO_PROCESO=""
        
    excel=openpyxl.load_workbook("02 Ch Genes ordenados con vecinos_PROTOTIPO.xlsx") 
    sheet=excel['Hoja1']
    sheet.cell(row=n,column=2).value = Entry 
    sheet.cell(row=n,column=3).value = Entry_name 
    sheet.cell(row=n,column=4).value = Protein 
    sheet.cell(row=n,column=5).value = Gen_Abrev 
    sheet.cell(row=n,column=6).value = ORGANISM_GEN 
    sheet.cell(row=n,column=7).value = ESTADO 
    sheet.cell(row=n,column=8).value = GO_PROCESO 
    excel.save("02 Ch Genes ordenados con vecinos_PROTOTIPO.xlsx") 
    n=n+1 